In [5]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from llm import generate_responses, generate_prompt
import json, re
from utils import load_pkl

cwd = os.getcwd()


In [25]:
filepath = os.path.join(cwd, "../../data", "easdrl/cooking_labeled_text_data.pkl")
indata = load_pkl(filepath)

words = indata[0]['words']
sents = indata[0]['sents']
for sent in sents:
    sentence = " ".join(sent) + "."
    print(sentence)
acts = indata[0]['acts']
paragraph = " ".join(words)
print(paragraph)
for act in acts:
    act_idx = act['act_idx']
    obj_idxs = act['obj_idxs'][0]
    act_type = act['act_type']
    related_acts = act['related_acts']
    
    act_name = words[act_idx]
    obj_names = " / ".join([words[i] for i in obj_idxs])
    related_acts_names = ", ".join([words[i] for i in related_acts])
    print(f"Action: {act_name} \tType: {act_type}, Related Acts: {related_acts_names} \n\tObject: {obj_names}")

MAKE breakfast the night before with a hash brown casserole.
Start by mixing together your cream of mushroom condensed soup sour cream salt pepper onion and cheese in a large bowl.
After these are well combined add in the hash browns and mix gently.
Spray your slow cooker with some non-stick cooking spray or coat with butter.
This will prevent sticking so you don't have a huge mess to clean up.
Pour the hash brown mixture into your slow cooker and put the cover on.
Turn the slow cooker on high and leave for 1 to 1.5 hours.
After this turn the heat to low and leave it on for 2.5 hours longer.
Turn off slow cooker and serve right away.
Keep on warm if you plan on waiting to enjoy for a little bit longer.
It's best when served with scrambled eggs.
If you want to add something unique to this recipe add your favorite vegetables.
It's generally best to use soft vegetables or those that have been slightly cooked before you add them.
This will ensure they don't turn out hard when the rest of t

In [9]:
verb_arg_prompt = generate_prompt("verb_arg", {"nl": paragraph})
print(verb_arg_prompt)



            Extract all verbs and verb phrases from the following natural language plan description.  
            For each verb or verb phrase, identify its associated arguments (e.g., subject, object, time, location, etc.) as nouns or noun phrases from the text.  
            
            Rules:
            - If a verb has no arguments, return an empty list for "arguments".
            - Use the lemma form for verbs (e.g., "running" -> "run"). Do not invent or infer verbs not explicitly mentioned in the text.
            - Use the lemma or canonical form for arguments (e.g., "boxes" -> "box"). Do not invent or infer arguments not explicitly mentioned in the text.
            - Each argument must denote one distinct entity or noun phrase; do not merge multiple entities into one argument
            - If an argument is a pronoun (e.g., "it", "them"), resolve it to the most recent explicit noun or noun phrase mentioned in the text.
            - Maintain the order of verbs as they appe

In [10]:
verb_arg_res = generate_responses("gpt-4o-mini",verb_arg_prompt)


In [11]:
resp = verb_arg_res['content'].strip()
m = re.search(r"```(?:json|jsonc)?\s*([\s\S]*?)\s*```", resp, re.I)
payload = m.group(1).strip() if m else resp
obj = json.loads(payload)

print(json.dumps(obj, indent=2))


[
  {
    "verb": "make",
    "arguments": [
      "breakfast",
      "the night before"
    ]
  },
  {
    "verb": "start",
    "arguments": []
  },
  {
    "verb": "mix",
    "arguments": [
      "cream of mushroom condensed soup",
      "sour cream",
      "salt",
      "pepper",
      "onion",
      "cheese",
      "a large bowl"
    ]
  },
  {
    "verb": "add",
    "arguments": [
      "the hash browns"
    ]
  },
  {
    "verb": "mix",
    "arguments": [
      "gently"
    ]
  },
  {
    "verb": "spray",
    "arguments": [
      "your slow cooker",
      "some non-stick cooking spray"
    ]
  },
  {
    "verb": "coat",
    "arguments": [
      "with butter"
    ]
  },
  {
    "verb": "prevent",
    "arguments": [
      "sticking"
    ]
  },
  {
    "verb": "have",
    "arguments": [
      "a huge mess"
    ]
  },
  {
    "verb": "clean",
    "arguments": [
      "up"
    ]
  },
  {
    "verb": "pour",
    "arguments": [
      "the hash brown mixture",
      "your slow cooker"
  

In [27]:
for x in obj:
    print(x)

{'verb': 'make', 'arguments': ['breakfast', 'the night before']}
{'verb': 'start', 'arguments': []}
{'verb': 'mix', 'arguments': ['cream of mushroom condensed soup', 'sour cream', 'salt', 'pepper', 'onion', 'cheese', 'a large bowl']}
{'verb': 'add', 'arguments': ['the hash browns']}
{'verb': 'mix', 'arguments': ['gently']}
{'verb': 'spray', 'arguments': ['your slow cooker', 'some non-stick cooking spray']}
{'verb': 'coat', 'arguments': ['with butter']}
{'verb': 'prevent', 'arguments': ['sticking']}
{'verb': 'have', 'arguments': ['a huge mess']}
{'verb': 'clean', 'arguments': ['up']}
{'verb': 'pour', 'arguments': ['the hash brown mixture', 'your slow cooker']}
{'verb': 'put', 'arguments': ['the cover']}
{'verb': 'turn', 'arguments': ['the slow cooker', 'on high']}
{'verb': 'leave', 'arguments': ['for 1 to 1.5 hours']}
{'verb': 'turn', 'arguments': ['the heat', 'to low']}
{'verb': 'leave', 'arguments': ['it', 'on for 2.5 hours longer']}
{'verb': 'turn', 'arguments': ['off slow cooker']}


In [26]:
srl_res_detail = generate_responses("gpt-4o-mini", prompt_detail)

In [27]:
resp = srl_res_detail['content'].strip()
m = re.search(r"```(?:json|jsonc)?\s*([\s\S]*?)\s*```", resp, re.I)
payload = m.group(1).strip() if m else resp
obj = json.loads(payload)

print(json.dumps(obj, indent=2))

[
  {
    "predicate": "pick",
    "arguments": {
      "ARG1": "the vehicle v1",
      "AM-LOC": "location l1"
    }
  },
  {
    "predicate": "collect",
    "arguments": {
      "ARG1": "the package p1"
    }
  },
  {
    "predicate": "have",
    "arguments": {
      "ARG1": "a size c1",
      "ARG2": "a size c2"
    }
  },
  {
    "predicate": "drive",
    "arguments": {
      "ARG1": "the vehicle v1",
      "AM-LOC": "from location l1 to location l2"
    }
  },
  {
    "predicate": "arrive",
    "arguments": {
      "AM-LOC": "at location l2"
    }
  },
  {
    "predicate": "drop",
    "arguments": {
      "ARG1": "the package p1",
      "AM-LOC": "in the vehicle v1"
    }
  },
  {
    "predicate": "retain",
    "arguments": {
      "ARG1": "its size c1",
      "ARG2": "size c2"
    }
  },
  {
    "predicate": "drive",
    "arguments": {
      "ARG1": "the vehicle v1",
      "AM-LOC": "from location l2 to location l1"
    }
  },
  {
    "predicate": "reach",
    "arguments": {
    

In [8]:
nl_other = """Clean a cassette deck.
Remove the tape from the deck. 
Open up the tape compartment. 
Gather Isopropyl alcohol (70-80%) Q-tips and a lens cleaning cloth. 
Soak the Q-tip in the alcohol. 
Clean the heads.  
Press fast forward and rewind so that the heads move.
Clean the pinch roller.  
Press play so the roller is moving when you clean it.
Clean the capstan.
Using the lens cloth, wipe out any other areas that appear dusty or dirty. 
Insert your tape.
Rewind the tape to the start, then fast forward to the end.
Rewind and fast forward until the tape plays smoothly.
Enjoy your music!"""

prompt_other = generate_prompt("srl", {"nl": nl_other})


In [ ]:
srl_res_other = generate_responses("gpt-4o-mini", prompt_other)

resp = srl_res_other['content'].strip()
m = re.search(r"```(?:json|jsonc)?\s*([\s\S]*?)\s*```", resp, re.I)
payload = m.group(1).strip() if m else resp
obj = json.loads(payload)

print(json.dumps(obj, indent=2))

[
  {
    "predicate": "clean",
    "arguments": {
      "ARG1": "a cassette deck"
    }
  },
  {
    "predicate": "remove",
    "arguments": {
      "ARG1": "the tape",
      "AM-LOC": "from the deck"
    }
  },
  {
    "predicate": "open",
    "arguments": {
      "ARG1": "up the tape compartment"
    }
  },
  {
    "predicate": "gather",
    "arguments": {
      "ARG1": "Isopropyl alcohol (70-80%) Q-tips and a lens cleaning cloth"
    }
  },
  {
    "predicate": "soak",
    "arguments": {
      "ARG1": "the Q-tip",
      "AM-INS": "in the alcohol"
    }
  },
  {
    "predicate": "clean",
    "arguments": {
      "ARG1": "the heads"
    }
  },
  {
    "predicate": "press",
    "arguments": {
      "ARG1": "fast forward and rewind",
      "AM-TMP": "so that the heads move"
    }
  },
  {
    "predicate": "clean",
    "arguments": {
      "ARG1": "the pinch roller"
    }
  },
  {
    "predicate": "press",
    "arguments": {
      "ARG1": "play",
      "AM-TMP": "so the roller is moving

In [9]:
verb_arg_prompt_other = generate_prompt("verb_arg", {"nl": nl_other})


In [10]:
verb_arg_other_res = generate_responses("gpt-4o-mini",verb_arg_prompt_other)

In [11]:
resp = verb_arg_other_res['content'].strip()
m = re.search(r"```(?:json|jsonc)?\s*([\s\S]*?)\s*```", resp, re.I)
payload = m.group(1).strip() if m else resp
obj = json.loads(payload)

print(json.dumps(obj, indent=2))

[
  {
    "verb": "Clean",
    "arguments": {
      "ARG0": "you",
      "ARG1": "a cassette deck"
    }
  },
  {
    "verb": "Remove",
    "arguments": {
      "ARG0": "you",
      "ARG1": "the tape",
      "ARG2": "from the deck"
    }
  },
  {
    "verb": "Open up",
    "arguments": {
      "ARG0": "you",
      "ARG1": "the tape compartment"
    }
  },
  {
    "verb": "Gather",
    "arguments": {
      "ARG0": "you",
      "ARG1": "Isopropyl alcohol (70-80%) Q-tips and a lens cleaning cloth"
    }
  },
  {
    "verb": "Soak",
    "arguments": {
      "ARG0": "you",
      "ARG1": "the Q-tip",
      "ARG2": "in the alcohol"
    }
  },
  {
    "verb": "Clean",
    "arguments": {
      "ARG0": "you",
      "ARG1": "the heads"
    }
  },
  {
    "verb": "Press",
    "arguments": {
      "ARG0": "you",
      "ARG1": "fast forward and rewind",
      "ARG2": "so that the heads move"
    }
  },
  {
    "verb": "Clean",
    "arguments": {
      "ARG0": "you",
      "ARG1": "the pinch roller"
